In [ ]:
# 간단 수정 - 오타 수정
print("🔧 오타 수정 중...")

# 컬럼명 확인
print("📋 실제 컬럼명:")
score_cols = [col for col in master_df.columns if '점수' in col or '적합도' in col]
print(score_cols)

# TOP 30 다시 추출 (올바른 컬럼명으로)
top30_fixed = master_df.nlargest(30, '곱창집_적합도_점수')[
    ['상권_코드_명', '곱창집_적합도_점수', 'T1_야간유동인구', '한식_월매출', 
     '한식_점포수', 'C2_상권활성도', 'T1_점수', 'T2_점수', 'C1_점수', 'C2_점수', 'E1_점수']
].copy()

print("✅ TOP 30 재추출 완료!")

In [ ]:
print("🏆 곱창집 입지 TOP 10 (수정):")
print("-" * 60)
for i, (idx, row) in enumerate(top30_fixed.head(10).iterrows(), 1):
    print(f"{i:2d}. {row['상권_코드_명']:<25} "
          f"(적합도: {row['곱창집_적합도_점수']:.1f}점)")
    print(f"    🌙 야간유동: {row['T1_야간유동인구']:8,.0f}명 "
          f"💰 한식매출: {row['한식_월매출']:10,.0f}원 "
          f"🏪 한식점포: {row['한식_점포수']:2.0f}개")
    print(f"    📊 점수: T1({row['T1_점수']:.0f}) T2({row['T2_점수']:.0f}) "
          f"C1({row['C1_점수']:.0f}) C2({row['C2_점수']:.0f}) E1({row['E1_점수']:.0f})")
    print()

In [ ]:
# 🍖 곱창집 입지 분석 KPI 스코어링 (하이브리드 모델)
# 03_kpi_scoring_hybrid.ipynb
#
# 목적: 한식음식점 중심 + 상권 전반 활성도를 종합한 곱창집 최적 입지 선정
# 하이브리드 모델: 직접 경쟁 분석 + 상권 매력도 통합 평가
# ================================================================================

# 📚 라이브러리 임포트
# ================================================================
import pandas as pd
import numpy as np
from pathlib import Path
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# 🎯 분석 설정
font_family = "Arial, sans-serif"
current_time = datetime.now().strftime("%Y-%m-%d %H:%M:%S")

print("🍖 곱창집 입지 분석 KPI 스코어링 (하이브리드 모델) 시작!")
print(f"⏰ 시작 시간: {current_time}")
print("=" * 70)

# 📂 경로 설정
# ================================================================
DATA_PATH = Path("../data")
RAW_PATH = DATA_PATH / "raw"
PROCESSED_PATH = DATA_PATH / "processed"
OUTPUT_PATH = Path("../docs")

# 파일 확인
master_file = PROCESSED_PATH / "seoul_gopchang_master_clean.csv"
if not master_file.exists():
    print(f"❌ 마스터 파일이 없습니다: {master_file}")
    print("💡 먼저 02-1_data_cleaning.ipynb를 실행해주세요!")
else:
    print(f"✅ 마스터 파일 확인: {master_file.name}")

# 🗃️ 정제된 마스터 데이터 로드
# ================================================================
print("\n📂 정제된 마스터 데이터 로드")
print("-" * 40)

master_df = pd.read_csv(master_file)
print(f"📊 마스터 데이터: {master_df.shape}")
print(f"📋 주요 컬럼: {list(master_df.columns)[:5]}...")

# 추가로 매출 원본 데이터 로드 (한식음식점 분석용)
sales_raw = pd.read_csv(RAW_PATH / "sales/sales_all.csv", encoding='utf-8')
print(f"📊 매출 원본: {sales_raw.shape}")

# 최신 분기 데이터만 추출
latest_quarter = sales_raw['기준_년분기_코드'].max()
sales_latest = sales_raw[sales_raw['기준_년분기_코드'] == latest_quarter].copy()
print(f"📅 최신 분기 ({latest_quarter}): {sales_latest.shape}")

print("=" * 70)

# 🍖 한식음식점 데이터 추출 및 분석
# ================================================================
print("\n🍖 한식음식점 데이터 분석 (곱창집 직접 경쟁 업종)")
print("-" * 50)

# 한식음식점 데이터 필터링
korean_food = sales_latest[sales_latest['서비스_업종_코드_명'] == '한식음식점'].copy()
print(f"📊 한식음식점 데이터: {korean_food.shape}")

# 상권별 한식음식점 지표 계산
korean_stats = korean_food.groupby('상권_코드').agg({
    '당월_매출_금액': 'sum',
    '당월_매출_건수': 'sum',
    '주중_매출_금액': 'sum', 
    '주말_매출_금액': 'sum',
    '월요일_매출_금액': 'sum',
    '화요일_매출_금액': 'sum',
    '수요일_매출_금액': 'sum',
    '목요일_매출_금액': 'sum',
    '금요일_매출_금액': 'sum',
    '토요일_매출_금액': 'sum',
    '일요일_매출_금액': 'sum'
}).reset_index()

# 컬럼명 변경
korean_stats.columns = [
    '상권_코드', '한식_월매출', '한식_월건수', '한식_주중매출', '한식_주말매출',
    '한식_월요일', '한식_화요일', '한식_수요일', '한식_목요일', 
    '한식_금요일', '한식_토요일', '한식_일요일'
]

# 한식음식점 점포 수 (경쟁밀도)
korean_count = korean_food.groupby('상권_코드').size().reset_index(name='한식_점포수')

# 한식음식점 통계 병합
korean_final = korean_stats.merge(korean_count, on='상권_코드', how='left')

print(f"📊 한식음식점 상권 수: {len(korean_final)}")
print(f"📋 한식음식점 주요 지표:")
for col in ['한식_월매출', '한식_점포수', '한식_주말매출']:
    if col in korean_final.columns:
        print(f"   • {col}: 평균 {korean_final[col].mean():,.0f}")

print("=" * 70)

# 🔗 마스터 데이터에 한식음식점 정보 조인
# ================================================================
print("\n🔗 마스터 데이터에 한식음식점 정보 조인")
print("-" * 40)

# 상권_코드를 기준으로 조인
master_df = master_df.merge(korean_final, on='상권_코드', how='left')

# 결측치 처리 (한식음식점이 없는 상권은 0으로)
korean_cols = [col for col in master_df.columns if col.startswith('한식_')]
for col in korean_cols:
    master_df[col] = master_df[col].fillna(0)

print(f"📊 조인 후 데이터: {master_df.shape}")
print(f"📈 한식음식점 정보 매칭률: {(master_df['한식_월매출'] > 0).sum() / len(master_df):.1%}")

print("=" * 70)

# 🎯 하이브리드 KPI 지표 계산
# ================================================================
print("\n🎯 하이브리드 KPI 지표 계산")
print("-" * 30)

# 1. T1: 야간 유동인구 지표 (30%)
print("1️⃣ T1: 야간 유동인구 지표 (30%)")
master_df['T1_야간유동인구'] = master_df['야간_유동인구_17_24'].fillna(0)
print(f"   📊 평균 야간 유동인구: {master_df['T1_야간유동인구'].mean():,.0f}명")

# 2. T2: 한식음식점 매출 밀도 (25%) - 곱창집 직접 경쟁
print("2️⃣ T2: 한식음식점 매출 밀도 (25%)")
master_df['T2_한식매출밀도'] = master_df['한식_월매출'] / (master_df['T1_야간유동인구'] + 1)
print(f"   📊 평균 한식 매출밀도: {master_df['T2_한식매출밀도'].mean():,.0f}원/명")

# 3. C1: 한식음식점 경쟁밀도 (20%) - 역수로 계산 (적을수록 좋음)
print("3️⃣ C1: 한식음식점 경쟁밀도 (20%)")
master_df['C1_경쟁밀도'] = 1 / (master_df['한식_점포수'] + 1)  # +1로 0 방지
print(f"   📊 평균 한식 점포수: {master_df['한식_점포수'].mean():.1f}개")
print(f"   📊 평균 경쟁밀도 점수: {master_df['C1_경쟁밀도'].mean():.3f}")

# 4. C2: 전체 상권 활성도 (15%) - 전체 매출 규모
print("4️⃣ C2: 전체 상권 활성도 (15%)")
master_df['C2_상권활성도'] = master_df['총_매출'].fillna(0)
print(f"   📊 평균 전체 매출: {master_df['C2_상권활성도'].mean():,.0f}원")

# 5. E1: 주류친화 지수 (10%) - 주말/야간 특화
print("5️⃣ E1: 주류친화 지수 (10%)")
# 한식음식점의 주말 vs 주중 비율
weekend_ratio = (master_df['한식_주말매출'] + 1) / (master_df['한식_주중매출'] + 1)
night_ratio = master_df['T1_야간유동인구'] / (master_df['총_유동인구_수'] + 1)
master_df['E1_주류친화지수'] = weekend_ratio * night_ratio
print(f"   📊 평균 주류친화 지수: {master_df['E1_주류친화지수'].mean():.3f}")

print("=" * 70)

# 📊 KPI 점수화 (0-100점)
# ================================================================
print("\n📊 KPI 점수화 (0-100점 척도)")
print("-" * 30)

def score_normalize(series, reverse=False):
    """0-100점으로 정규화 (reverse=True면 작을수록 높은 점수)"""
    if series.std() == 0:
        return pd.Series([50] * len(series), index=series.index)
    
    if reverse:
        # 작을수록 좋은 지표 (예: 경쟁밀도)
        scores = 100 - ((series - series.min()) / (series.max() - series.min()) * 100)
    else:
        # 클수록 좋은 지표
        scores = (series - series.min()) / (series.max() - series.min()) * 100
    
    return scores.fillna(50)

# 각 지표별 점수화
master_df['T1_점수'] = score_normalize(master_df['T1_야간유동인구'])
master_df['T2_점수'] = score_normalize(master_df['T2_한식매출밀도'])
master_df['C1_점수'] = score_normalize(master_df['한식_점포수'], reverse=True)  # 적을수록 좋음
master_df['C2_점수'] = score_normalize(master_df['C2_상권활성도'])
master_df['E1_점수'] = score_normalize(master_df['E1_주류친화지수'])

# 점수 요약
for i, (code, name, weight) in enumerate([
    ('T1', '야간유동인구', 30),
    ('T2', '한식매출밀도', 25), 
    ('C1', '경쟁밀도', 20),
    ('C2', '상권활성도', 15),
    ('E1', '주류친화지수', 10)
], 1):
    score_col = f'{code}_점수'
    avg_score = master_df[score_col].mean()
    print(f"{i}️⃣ {code}: {name} (가중치 {weight}%) - 평균 {avg_score:.1f}점")

print("=" * 70)

# 🏆 최종 곱창집 적합도 점수 계산
# ================================================================
print("\n🏆 최종 곱창집 적합도 점수 계산")
print("-" * 35)

# 가중치 적용한 최종 점수
weights = {
    'T1_점수': 0.30,  # 야간 유동인구 (30%)
    'T2_점수': 0.25,  # 한식 매출밀도 (25%)
    'C1_점수': 0.20,  # 경쟁밀도 (20%)
    'C2_점수': 0.15,  # 상권활성도 (15%)
    'E1_점수': 0.10   # 주류친화지수 (10%)
}

master_df['곱창집_적합도_점수'] = (
    master_df['T1_점수'] * weights['T1_점수'] +
    master_df['T2_점수'] * weights['T2_점수'] +
    master_df['C1_점수'] * weights['C1_점수'] +
    master_df['C2_점수'] * weights['C2_점수'] +
    master_df['E1_점수'] * weights['E1_점수']
)

print(f"📊 최종 적합도 점수:")
print(f"   평균: {master_df['곱창집_적합도_점수'].mean():.1f}점")
print(f"   최고: {master_df['곱창집_적합도_점수'].max():.1f}점")
print(f"   최저: {master_df['곱창집_적합도_점수'].min():.1f}점")
print(f"   표준편차: {master_df['곱창집_적합도_점수'].std():.1f}점")

print("=" * 70)

# 🥇 TOP 30 곱창집 최적 입지 상권
# ================================================================
print("\n🥇 TOP 30 곱창집 최적 입지 상권")
print("-" * 35)

# TOP 30 추출
top30 = master_df.nlargest(30, '곱창집_적합도_점수')[
    ['상권_코드_명', '곱창집_적합도_점수', 'T1_야간유동인구', '한식_월매출', 
     '한식_점포수', 'C2_상권활성도', 'T1_점수', 'T2_점수', 'C1_점수', 'C2_점수', 'E1_점수']
].copy()

print("🏆 곱창집 입지 TOP 30:")
print("-" * 60)
for i, (idx, row) in enumerate(top30.iterrows(), 1):
    print(f"{i:2d}. {row['상권_코드_명']:<25} "
          f"(적합도: {row['곱창집_적합도_점수']:.1f}점)")
    print(f"    🌙 야간유동: {row['T1_야간유동인구']:8,.0f}명 "
          f"💰 한식매출: {row['한식_월매출']:10,.0f}원 "
          f"🏪 한식점포: {row['한식_점포수']:2.0f}개")
    if i <= 10:  # TOP 10만 상세 표시
        print(f"    📊 점수: T1({row['T1_점수']:.0f}) T2({row['T2_점수']:.0f}) "
              f"C1({row['C1_점수']:.0f}) C2({row['C2_점수']:.0f}) E1({row['E1_점수']:.0f})")
    print()

print("=" * 70)

# 📊 KPI 분석 시각화
# ================================================================
print("\n📊 KPI 분석 시각화")
print("-" * 20)

# 1. TOP 30 레이더 차트
fig_radar = go.Figure()

# TOP 10의 평균 점수
top10_avg = top30.head(10)[['T1_점수', 'T2_점수', 'C1_점수', 'C2_점수', 'E1_점수']].mean()
all_avg = master_df[['T1_점수', 'T2_점수', 'C1_점수', 'C2_점수', 'E1_점수']].mean()

categories = ['야간유동인구', '한식매출밀도', '경쟁우위성', '상권활성도', '주류친화도']

fig_radar.add_trace(go.Scatterpolar(
    r=list(top10_avg) + [top10_avg.iloc[0]], 
    theta=categories + [categories[0]],
    fill='toself',
    name='TOP 10 평균',
    line_color='red'
))

fig_radar.add_trace(go.Scatterpolar(
    r=list(all_avg) + [all_avg.iloc[0]],
    theta=categories + [categories[0]],
    fill='toself',
    name='전체 평균',
    line_color='blue',
    opacity=0.6
))

fig_radar.update_layout(
    title="🍖 TOP 10 vs 전체 평균 KPI 비교 (레이더 차트)",
    polar=dict(
        radialaxis=dict(
            visible=True,
            range=[0, 100]
        )),
    font=dict(family=font_family, size=12),
    height=500
)

fig_radar.show()

# 2. TOP 30 적합도 점수 분포
fig_top30 = px.bar(
    top30.head(15), 
    x='상권_코드_명', 
    y='곱창집_적합도_점수',
    title="🏆 곱창집 입지 TOP 15 적합도 점수",
    color='곱창집_적합도_점수',
    color_continuous_scale='Reds'
)

fig_top30.update_layout(
    xaxis_title="상권명",
    yaxis_title="적합도 점수",
    font=dict(family=font_family, size=12),
    height=500,
    xaxis={'tickangle': 45}
)

fig_top30.show()

# 3. KPI 가중치별 기여도 분석
weights_data = pd.DataFrame({
    'KPI': ['야간유동인구', '한식매출밀도', '경쟁우위성', '상권활성도', '주류친화도'],
    '가중치': [30, 25, 20, 15, 10],
    '설명': ['17-24시 유동인구', '곱창집 직접경쟁 매출', '한식점포 수 (역수)', '전체 상권 매출', '주말/야간 특화도']
})

fig_weights = px.pie(
    weights_data, 
    values='가중치', 
    names='KPI',
    title="🎯 곱창집 적합도 KPI 가중치 구성",
    color_discrete_sequence=px.colors.qualitative.Set3
)

fig_weights.update_traces(textposition='inside', textinfo='percent+label')
fig_weights.update_layout(
    font=dict(family=font_family, size=12),
    height=500
)

fig_weights.show()

# 4. 야간 유동인구 vs 한식 매출 상관관계
fig_scatter = px.scatter(
    master_df, 
    x='T1_야간유동인구', 
    y='한식_월매출',
    size='곱창집_적합도_점수',
    color='곱창집_적합도_점수',
    hover_name='상권_코드_명',
    title="🌙 야간 유동인구 vs 한식음식점 매출 상관관계",
    labels={
        'T1_야간유동인구': '야간 유동인구 (명)',
        '한식_월매출': '한식음식점 월매출 (원)',
        '곱창집_적합도_점수': '적합도 점수'
    },
    color_continuous_scale='Reds'
)

fig_scatter.update_layout(
    font=dict(family=font_family, size=12),
    height=500
)

fig_scatter.show()

print("=" * 70)

# 💾 결과 저장
# ================================================================
print("\n💾 분석 결과 저장")
print("-" * 15)

# CSV 저장
output_csv = OUTPUT_PATH / "gopchang_kpi_analysis.csv"
master_df.to_csv(output_csv, index=False, encoding='utf-8-sig')
print(f"✅ 전체 결과: {output_csv.name}")

# TOP 30 CSV 저장
top30_csv = OUTPUT_PATH / "gopchang_top30_locations.csv"
top30.to_csv(top30_csv, index=False, encoding='utf-8-sig')
print(f"✅ TOP 30: {top30_csv.name}")

# 텍스트 보고서 저장
report_file = OUTPUT_PATH / "gopchang_kpi_report.txt"
with open(report_file, 'w', encoding='utf-8') as f:
    f.write("🍖 곱창집 입지 분석 KPI 스코어링 보고서\n")
    f.write(f"📅 분석일시: {current_time}\n")
    f.write("=" * 50 + "\n\n")
    
    f.write("📊 하이브리드 KPI 모델 구성:\n")
    f.write("• T1: 야간유동인구 (30%) - 17-24시 유동인구\n")
    f.write("• T2: 한식매출밀도 (25%) - 곱창집 직접경쟁 매출\n") 
    f.write("• C1: 경쟁우위성 (20%) - 한식점포 수 (역수)\n")
    f.write("• C2: 상권활성도 (15%) - 전체 상권 매출 규모\n")
    f.write("• E1: 주류친화도 (10%) - 주말/야간 특화 지수\n\n")
    
    f.write("🏆 TOP 10 곱창집 최적 입지:\n")
    for i, (idx, row) in enumerate(top30.head(10).iterrows(), 1):
        f.write(f"{i:2d}. {row['상권_코드_명']} (적합도: {row['곱창집_적합도_점수']:.1f}점)\n")
    
    f.write(f"\n📈 분석 요약:\n")
    f.write(f"• 전체 분석 상권: {len(master_df):,}개\n")
    f.write(f"• 평균 적합도 점수: {master_df['곱창집_적합도_점수'].mean():.1f}점\n")
    f.write(f"• 한식음식점 보유 상권: {(master_df['한식_점포수'] > 0).sum():,}개\n")

print(f"✅ 보고서: {report_file.name}")

print("=" * 70)

# 🎯 분석 완료 요약
# ================================================================
print("\n🎯 곱창집 입지 분석 KPI 스코어링 완료!")
print("-" * 40)

end_time = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
print(f"✅ 완료 시간: {end_time}")

print(f"\n📊 최종 분석 결과:")
print(f"   🎯 하이브리드 KPI 모델: 5개 지표")
print(f"   🏆 TOP 30 최적 입지 선정 완료")
print(f"   📈 인터랙티브 시각화: 4개 차트")
print(f"   💾 결과 파일: 3개 저장")

print(f"\n🍖 곱창집 특화 인사이트:")
print(f"   • 최적 1위: {top30.iloc[0]['상권_코드_명']} ({top30.iloc[0]['곱창집_적합도_점수']:.1f}점)")
print(f"   • 한식 경쟁 고려: 매출밀도 + 점포수 균형")
print(f"   • 야간 특화 강조: 17-24시 유동인구 30% 반영")

print(f"\n🚀 다음 단계 옵션:")
print(f"   A. 지도 시각화 (TOP 30 위치 매핑)")
print(f"   B. 세부 분석 (TOP 10 심층 분석)")
print(f"   C. 실시간 대시보드 구축")
print(f"   D. 최종 보고서 작성")

print("=" * 70)
print("🍖 하이브리드 KPI 스코어링 완료!")
print("🎯 이제 데이터 기반 곱창집 최적 입지 30곳을 확보했습니다!")